<h2>Basic SQL Syntax</h2>
<p>
Many people are already familiar with SQL syntax.  Once your data is organized in a dataframe, you can create what's called a view of the data frame.  The view will allow you to write SQL statements view the spark objects sql method.
</p>

In [1]:
from pyspark.sql import SparkSession
from datetime import datetime
path = "/Users/josephgartner/Desktop/data/"

spk = SparkSession.builder.master("local").getOrCreate()
df = spk.read.json(path)

df.createTempView("tweets")

<p>Walking through the basic data slicing as in the previous notebook, we can perform all the same actions as before, such as:</p>

In [2]:
#Selecting specific fields
df2 = spk.sql("SELECT lang, user.name FROM tweets")
df2.show(5)

+----+------------+
|lang|        name|
+----+------------+
|  en| Ross Dillon|
|  en|ian slingsby|
|  en|       Nduka|
|  en|  Il Padrino|
|  en|Graham Banks|
+----+------------+
only showing top 5 rows



In [3]:
#Remove null fields
df2 = spk.sql("SELECT lang, user.name, geo FROM tweets WHERE geo != NULL")
df2.show(5)

+----+----+---+
|lang|name|geo|
+----+----+---+
+----+----+---+



In [4]:
#Find specific field values
df2 = spk.sql("SELECT lang, user.name FROM tweets WHERE lang = 'en'")
df2.show(5)

+----+------------+
|lang|        name|
+----+------------+
|  en| Ross Dillon|
|  en|ian slingsby|
|  en|       Nduka|
|  en|  Il Padrino|
|  en|Graham Banks|
+----+------------+
only showing top 5 rows



In [5]:
#Perform basic transformations
df2 = spk.sql("SELECT geo.coordinates[0], geo.coordinates[0] + 1 FROM tweets WHERE geo != NULL")
df2.show(5)

+-----------------------------------+---------------------------------------------------------+
|geo.coordinates AS `coordinates`[0]|(geo.coordinates AS `coordinates`[0] + CAST(1 AS DOUBLE))|
+-----------------------------------+---------------------------------------------------------+
+-----------------------------------+---------------------------------------------------------+



In [6]:
#And basic groupings
df2 = spk.sql("SELECT lang, COUNT(*) FROM tweets GROUP BY lang")
df2.show(10)

+----+--------+
|lang|count(1)|
+----+--------+
|  en|   97773|
|  vi|       5|
|  ne|       3|
|  ps|       1|
|  ro|      44|
|  sl|       9|
| und|    7162|
|  ur|      56|
|  lv|      13|
|  pl|     116|
+----+--------+
only showing top 10 rows



<h2>UDFs</h2>
<p>In order to do more sophisiticated selection, one will opt for a user defined function.  We will start by showing that the function works locally on data from the sample</p>

In [7]:
def point_in_box(geo):
    if geo is None or geo.coordinates is None:
        return False
    if geo.coordinates[0] > 53 or geo.coordinates[0] < 52:
        return False
    if geo.coordinates[1] > -1 or geo.coordinates[1] < -2:
        return False
    return True

In [8]:
df2 = spk.sql("SELECT * FROM tweets WHERE geo != NULL")
temp = df2.take(5)

for tweet in temp:
    print point_in_box(tweet.geo)

<p>When you try to use it in a select statement, it fails</p>

In [9]:
df2 = spk.sql("SELECT * FROM tweets WHERE point_in_box(geo)")

AnalysisException: u"Undefined function: 'point_in_box'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 27"

In [10]:
from pyspark.sql.types import BooleanType

sqlContext.registerFunction("dist_pib", lambda geo: point_in_box(geo), returnType=BooleanType())

In [12]:
df2 = spk.sql("SELECT lang, user.name, geo FROM tweets WHERE dist_pib(geo)")
df2.show(5)

+----+----------+--------------------+
|lang|      name|                 geo|
+----+----------+--------------------+
|  in|      amar|[WrappedArray(52....|
|  en|John Coyle|[WrappedArray(52....|
|  en| Tarnya 🌱|[WrappedArray(52....|
|  en|   almajir|[WrappedArray(52....|
|  en|John Coyle|[WrappedArray(52....|
+----+----------+--------------------+
only showing top 5 rows

